In [161]:
import duckdb
import re
import os
import dotenv
dotenv.load_dotenv(".env")

False

In [162]:
# dataset link https://www.kaggle.com/datasets/hetbabariya/imdb-movies-data-collection-5000-records?resource=download
db = duckdb.connect()
db.execute("CREATE TABLE imdb AS SELECT * FROM read_csv_auto('IMDB_Movies_Dataset.csv')")
db.execute("ALTER TABLE imdb DROP COLUMN IF EXISTS column00") # incremental name for empty columns  (column01, 02, ...)
df = db.sql("SELECT * FROM imdb")
# df.df() # pandas df
df # string

┌────────────────────────────────────────────┬────────────────┬────────────────────────────────────┬───────────────────────────────────────────────────┬───────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────┬──────────────────────────────────────┬────────────────────────────────────────┬──────────────────────────┬─────────────────┬────────────────────┐
│                   Title                    │ Average Rating │              Director              │                      Writer                       │ Metascore │                                                                                                                                                     Cast                                   

In [163]:
db.execute("ALTER TABLE imdb RENAME COLUMN \"Average Rating\" TO Rating")
db.sql("SELECT * FROM imdb").columns

['Title',
 'Rating',
 'Director',
 'Writer',
 'Metascore',
 'Cast',
 'Release Date',
 'Country of Origin',
 'Languages',
 'Budget',
 'Worldwide Gross',
 'Runtime']

In [164]:
db.sql("SHOW imdb")

┌───────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name    │ column_type │  null   │   key   │ default │  extra  │
│      varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ Title             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Rating            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ Director          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Writer            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Metascore         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ Cast              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Release Date      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Country of Origin │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Languages         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Budget    

In [165]:
# DuckDB's implementation is based on PostgreSQL's information schema.
db.sql("""
SELECT * 
    FROM information_schema.columns 
""").df()

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,memory,main,imdb,Title,1,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
1,memory,main,imdb,Rating,2,None,YES,DOUBLE,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
2,memory,main,imdb,Director,3,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
3,memory,main,imdb,Writer,4,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
4,memory,main,imdb,Metascore,5,None,YES,DOUBLE,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
5,memory,main,imdb,Cast,6,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
6,memory,main,imdb,Release Date,7,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
7,memory,main,imdb,Country of Origin,8,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
8,memory,main,imdb,Languages,9,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
9,memory,main,imdb,Budget,10,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None


In [166]:
db.sql("SUMMARIZE imdb").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,Title,VARCHAR,#Home,色，戒,5068,None,None,None,None,None,4989,0.00
1,Rating,DOUBLE,7.2,9.3,17,7.632892363198688,0.3652432599971371,7.303231872135386,7.56332647663826,7.846631097560976,4989,0.00
2,Director,VARCHAR,A.L. Vijay,Özer Kiziltan,2707,None,None,None,None,None,4989,0.00
3,Writer,VARCHAR,"A.A. Milne, Ernest Shepard, Alex Ross Perry",Ümit Ünal,4347,None,None,None,None,None,4989,0.02
4,Metascore,DOUBLE,25.0,100.0,97,74.68445171849427,12.298910502972529,67.0,75.68220914440923,83.47860101242523,4989,38.77
5,Cast,VARCHAR,"'Freeway' Ricky Ross, Susan Sarandon, David Si...","Éric Cantona, Alex Ferguson, Manchester United...",4347,None,None,None,None,None,4989,0.14
6,Release Date,VARCHAR,1928 (Italy),"September 9, 2020 (India)",4693,None,None,None,None,None,4989,0.00
7,Country of Origin,VARCHAR,"Afghanistan, Ireland, Japan, Netherlands, Iran","Yugoslavia, United States",873,None,None,None,None,None,4989,0.06
8,Languages,VARCHAR,"Aboriginal, Japanese, Hokkien","Zulu, Sotho, Xhosa, Afrikaans, English",1109,None,None,None,None,None,4989,0.42
9,Budget,VARCHAR,"$1,000,000 (estimated)","₹991,000 (estimated)",1110,None,None,None,None,None,4989,46.86


In [167]:

db.execute(
    "SELECT Runtime, COUNT(*) AS count FROM imdb GROUP BY Runtime ORDER BY count DESC"
).df()

,Runtime,count
0,1 hour 35 minutes,109
1,1 hour 50 minutes,109
2,1 hour 43 minutes,100
3,1 hour 58 minutes,94
4,1 hour 40 minutes,92
...,...,...
177,1 hour 3 minutes,1
178,3 hours 10 minutes,1
179,51 minutes,1
180,3 hours 19 minutes,1


In [168]:
def runtime_to_minutes(string: str) -> int:
    hours = re.search(r"(\d+) hours?", string)
    if hours:
        hours = int(hours.group(1))
    else:
        hours = 0
    minutes = re.search(r"(\d+) minutes", string)
    if minutes:
        minutes = int(minutes.group(1))
    else:
        minutes = 0
    return hours * 60 + minutes


runtime_to_minutes("2 hours 25 minutes")

145

In [169]:
db.create_function("runtime_to_minutes", runtime_to_minutes)

In [170]:
df = db.sql("SELECT *, runtime_to_minutes(Runtime) as 'Runtime (minutes)' FROM imdb")
df.df()


,Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes)
0,The Shawshank Redemption,9.3,Frank Darabont,"Stephen King, Frank Darabont",82.0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","October 14, 1994 (India)",United States,English,"$25,000,000 (estimated)","$29,332,133",2 hours 22 minutes,142
1,Attack on Titan the Movie: The Last Attack,9.3,Yûichirô Hayashi,"Natsuki Hanae, Yoshimasa Hosoya, Marina Inoue",NaN,"Natsuki Hanae, Yoshimasa Hosoya, Marina Inoue,...","November 8, 2024 (Japan)",Japan,Japanese,None,"$3,513,659",2 hours 25 minutes,145
2,The Godfather,9.2,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola",100.0,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1978 (India),United States,"English, Italian, Latin","$6,000,000 (estimated)","$250,342,198",2 hours 55 minutes,175
3,Hababam Sinifi,9.2,Ertem Egilmez,"Umur Bugay, Rifat Ilgaz",NaN,"Münir Özkul, Tarik Akan, Halit Akçatepe, Kemal...","April 1, 1975 (Turkey)",Turkey,Turkish,None,None,1 hour 25 minutes,85
4,Ramayana: The Legend of Prince Rama,9.2,"Ram Mohan, Yûgô Sakô, Koichi Saski","Rani Burra, Ram Mohan, Yûgô Sakô",NaN,"Arun Govil, Nikhil Kapoor, Edie Mirman, Rael P...","September 25, 2001 (India)","India, Japan",English,None,None,2 hours 15 minutes,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,La comunidad,7.2,Álex de la Iglesia,"Jorge Guerricaechevarría, Álex de la Iglesia",NaN,"Carmen Maura, Eduardo Antuña, María Asquerino,...","September 29, 2000 (Spain)",Spain,Spanish,"€4,000,000 (estimated)","$290,754",1 hour 50 minutes,110
4985,The House of Fear,7.2,Roy William Neill,"Roy Chanslor, Arthur Conan Doyle",NaN,"Basil Rathbone, Nigel Bruce, Aubrey Mather, De...","March 16, 1945 (United States)",United States,English,None,None,1 hour 9 minutes,69
4986,Kurôzu zero II,7.2,Takashi Miike,"Hiroshi Takahashi, Shôgo Mutô",NaN,"Shun Oguri, Kyôsuke Yabe, Meisa Kuroki, Kenta ...","April 11, 2009 (Japan)",Japan,Japanese,None,"$29,893,636",2 hours 13 minutes,133
4987,La cage dorée,7.2,Ruben Alves,"Ruben Alves, Luc-Olivier Veuve, Jean-André Yerlès",NaN,"Rita Blanco, Joaquim de Almeida, Roland Giraud...","April 24, 2013 (France)",France,"French, Portuguese, English","€7,000,000 (estimated)","$17,871,370",1 hour 30 minutes,90


In [171]:
# to add column permanently
db.execute("ALTER TABLE imdb ADD COLUMN \"Runtime (minutes)\" INT")
db.execute("UPDATE imdb SET \"Runtime (minutes)\" = runtime_to_minutes(Runtime)")
db.sql("SHOW imdb")


┌───────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name    │ column_type │  null   │   key   │ default │  extra  │
│      varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ Title             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Rating            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ Director          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Writer            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Metascore         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ Cast              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Release Date      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Country of Origin │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Languages         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Budget    

In [172]:
def money_to_usd(string: str | None) -> int | None:
    if not isinstance(string, str) or not "$" in string:
        return None
    budget = int(re.sub(r"[^\d]", "", string))
    return budget
money_to_usd("$6,000,000 (estimated)")

6000000

In [173]:
# db.remove_function("money_to_usd") # I'm tired from errors that something already exists. It's hard to simply override something
db.create_function("money_to_usd", money_to_usd, null_handling="SPECIAL")

In [174]:
df = db.sql('''
    with budgets as (SELECT *, 
    money_to_usd(Budget) as "Budget (usd)",
    money_to_usd("Worldwide Gross") as "Worldwide Gross (usd)"
    FROM imdb)
    select *, "Worldwide Gross (usd)" - "Budget (usd)" from budgets
'''
)
df.df()




,Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd),"(""Worldwide Gross (usd)"" - ""Budget (usd)"")"
0,The Shawshank Redemption,9.3,Frank Darabont,"Stephen King, Frank Darabont",82.0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","October 14, 1994 (India)",United States,English,"$25,000,000 (estimated)","$29,332,133",2 hours 22 minutes,142,25000000.0,29332133.0,4332133.0
1,Attack on Titan the Movie: The Last Attack,9.3,Yûichirô Hayashi,"Natsuki Hanae, Yoshimasa Hosoya, Marina Inoue",NaN,"Natsuki Hanae, Yoshimasa Hosoya, Marina Inoue,...","November 8, 2024 (Japan)",Japan,Japanese,None,"$3,513,659",2 hours 25 minutes,145,NaN,3513659.0,NaN
2,The Godfather,9.2,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola",100.0,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1978 (India),United States,"English, Italian, Latin","$6,000,000 (estimated)","$250,342,198",2 hours 55 minutes,175,6000000.0,250342198.0,244342198.0
3,Hababam Sinifi,9.2,Ertem Egilmez,"Umur Bugay, Rifat Ilgaz",NaN,"Münir Özkul, Tarik Akan, Halit Akçatepe, Kemal...","April 1, 1975 (Turkey)",Turkey,Turkish,None,None,1 hour 25 minutes,85,NaN,NaN,NaN
4,Ramayana: The Legend of Prince Rama,9.2,"Ram Mohan, Yûgô Sakô, Koichi Saski","Rani Burra, Ram Mohan, Yûgô Sakô",NaN,"Arun Govil, Nikhil Kapoor, Edie Mirman, Rael P...","September 25, 2001 (India)","India, Japan",English,None,None,2 hours 15 minutes,135,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,La comunidad,7.2,Álex de la Iglesia,"Jorge Guerricaechevarría, Álex de la Iglesia",NaN,"Carmen Maura, Eduardo Antuña, María Asquerino,...","September 29, 2000 (Spain)",Spain,Spanish,"€4,000,000 (estimated)","$290,754",1 hour 50 minutes,110,NaN,290754.0,NaN
4985,The House of Fear,7.2,Roy William Neill,"Roy Chanslor, Arthur Conan Doyle",NaN,"Basil Rathbone, Nigel Bruce, Aubrey Mather, De...","March 16, 1945 (United States)",United States,English,None,None,1 hour 9 minutes,69,NaN,NaN,NaN
4986,Kurôzu zero II,7.2,Takashi Miike,"Hiroshi Takahashi, Shôgo Mutô",NaN,"Shun Oguri, Kyôsuke Yabe, Meisa Kuroki, Kenta ...","April 11, 2009 (Japan)",Japan,Japanese,None,"$29,893,636",2 hours 13 minutes,133,NaN,29893636.0,NaN
4987,La cage dorée,7.2,Ruben Alves,"Ruben Alves, Luc-Olivier Veuve, Jean-André Yerlès",NaN,"Rita Blanco, Joaquim de Almeida, Roland Giraud...","April 24, 2013 (France)",France,"French, Portuguese, English","€7,000,000 (estimated)","$17,871,370",1 hour 30 minutes,90,NaN,17871370.0,NaN


In [175]:
db.execute("""ALTER TABLE imdb ADD COLUMN "Budget (usd)" BIGINT""")
db.execute("""ALTER TABLE imdb ADD COLUMN "Worldwide Gross (usd)" BIGINT""")
db.execute(
    """UPDATE imdb SET "Budget (usd)" = money_to_usd("Budget"), "Worldwide Gross (usd)" = money_to_usd("Worldwide Gross")"""
)
db.execute("ALTER TABLE imdb ADD COLUMN \"Profit (usd)\" BIGINT")
db.execute("UPDATE imdb SET \"Profit (usd)\" = \"Worldwide Gross (usd)\" - \"Budget (usd)\"")

# I hate this so much. It was an error about int and bigint so I needed to update type of the column. To do this I need to comment out alter statements (because columns already exists) and run
# db.execute("ALTER TABLE imdb ALTER COLUMN \"Profit (usd)\" TYPE BIGINT")

In [176]:
db.sql(
    "SELECT * FROM imdb ORDER BY \"Budget (usd)\" DESC LIMIT 10"
).df()


,Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd),Profit (usd)
0,Avengers: Endgame,8.4,"Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely, Stan Lee",78.0,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...","April 26, 2019 (India)",United States,"English, Japanese, Xhosa, German","$356,000,000 (estimated)","$2,799,439,100",3 hours 1 minute,180,356000000,2.799439e+09,2.443439e+09
1,The Little Mermaid,7.2,Rob Marshall,"David Magee, Hans Christian Andersen, John Musker",59.0,"Halle Bailey, Jonah Hauer-King, Melissa McCart...","May 26, 2023 (India)",United States,English,"$355,100,000 (estimated)","$569,626,289",2 hours 15 minutes,135,355100000,5.696263e+08,2.145263e+08
2,Avatar: The Way of Water,7.5,James Cameron,"James Cameron, Rick Jaffa, Amanda Silver",67.0,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","December 16, 2022 (India)",United States,English,"$350,000,000 (estimated)","$2,320,250,281",3 hours 12 minutes,192,350000000,2.320250e+09,1.970250e+09
3,Avengers: Infinity War,8.4,"Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely, Stan Lee",68.0,"Robert Downey Jr., Chris Hemsworth, Mark Ruffa...","April 27, 2018 (India)","United States, Norway",English,"$321,000,000 (estimated)","$2,052,415,039",2 hours 29 minutes,149,321000000,2.052415e+09,1.731415e+09
4,Zack Snyder's Justice League,7.9,Zack Snyder,"Jerry Siegel, Joe Shuster, Zack Snyder",54.0,"Henry Cavill, Ben Affleck, Gal Gadot, Amy Adam...","March 18, 2021 (India)","United States, Canada, New Zealand, United Kin...","English, Icelandic, French","$300,000,000 (estimated)",None,4 hours 2 minutes,242,300000000,NaN,NaN
5,Mission: Impossible - Dead Reckoning Part One,7.7,Christopher McQuarrie,"Bruce Geller, Christopher McQuarrie, Erik Jend...",81.0,"Tom Cruise, Hayley Atwell, Ving Rhames, Simon ...","July 12, 2023 (India)",United States,"English, French, Italian, Russian","$291,000,000 (estimated)","$570,619,838",2 hours 43 minutes,163,291000000,5.706198e+08,2.796198e+08
6,Tangled,7.7,"Nathan Greno, Byron Howard","Dan Fogelman, Jacob Grimm, Wilhelm Grimm",71.0,"Mandy Moore, Zachary Levi, Donna Murphy, Ron P...","January 21, 2011 (India)","United States, Japan","English, German","$260,000,000 (estimated)","$591,806,017",1 hour 40 minutes,100,260000000,5.918060e+08,3.318060e+08
7,The Dark Knight Rises,8.4,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer",78.0,"Christian Bale, Tom Hardy, Anne Hathaway, Gary...","July 20, 2012 (India)","United States, United Kingdom","English, Arabic","$250,000,000 (estimated)","$1,114,975,066",2 hours 44 minutes,164,250000000,1.114975e+09,8.649751e+08
8,Guardians of the Galaxy Vol. 3,7.9,James Gunn,"James Gunn, Jim Starlin, Stan Lee",64.0,"Chris Pratt, Chukwudi Iwuji, Bradley Cooper, P...","May 5, 2023 (India)","United States, New Zealand, France, Canada",English,"$250,000,000 (estimated)","$845,555,777",2 hours 30 minutes,150,250000000,8.455558e+08,5.955558e+08
9,Captain America: Civil War,7.8,"Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely, Joe Simon",75.0,"Chris Evans, Robert Downey Jr., Scarlett Johan...","May 6, 2016 (India)","United States, Germany","English, German, Xhosa, Russian, Romanian","$250,000,000 (estimated)","$1,155,046,416",2 hours 27 minutes,147,250000000,1.155046e+09,9.050464e+08


In [177]:
# you can use youplot for chart https://duckdb.org/docs/guides/data_viewers/youplot.html
# but easiest way is to convert to pandas df and use .plot

In [178]:
db.sql(
    "SELECT * FROM imdb WHERE \"Cast\" LIKE '%Johnny Depp%'"
).df()

,Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd),Profit (usd)
0,Pirates of the Caribbean: The Curse of the Bla...,8.1,Gore Verbinski,"Ted Elliott, Terry Rossio, Stuart Beattie",63.0,"Johnny Depp, Geoffrey Rush, Orlando Bloom, Kei...","August 15, 2003 (India)",United States,English,"$140,000,000 (estimated)","$654,264,546",2 hours 23 minutes,143,140000000.0,654264546,514264546.0
1,Platoon,8.1,Oliver Stone,Oliver Stone,92.0,"Charlie Sheen, Tom Berenger, Willem Dafoe, Kei...","February 13, 1989 (India)","United States, United Kingdom","English, Vietnamese","$6,000,000 (estimated)","$138,545,632",2 hours,120,6000000.0,138545632,132545632.0
2,Edward Scissorhands,7.9,Tim Burton,"Tim Burton, Caroline Thompson",74.0,"Johnny Depp, Winona Ryder, Dianne Wiest, Antho...","January 1, 1992 (India)",United States,English,"$20,000,000 (estimated)","$86,024,005",1 hour 45 minutes,105,20000000.0,86024005,66024005.0
3,Ed Wood,7.8,Tim Burton,"Rudolph Grey, Scott Alexander, Larry Karaszewski",71.0,"Johnny Depp, Martin Landau, Sarah Jessica Park...",1994 (India),United States,English,"$18,000,000 (estimated)","$5,888,045",2 hours 7 minutes,127,18000000.0,5888045,-12111955.0
4,What's Eating Gilbert Grape,7.7,Lasse Hallström,Peter Hedges,73.0,"Johnny Depp, Leonardo DiCaprio, Juliette Lewis...","March 4, 1994 (United States)",United States,English,"$11,000,000 (estimated)","$10,032,765",1 hour 58 minutes,118,11000000.0,10032765,-967235.0
5,Donnie Brasco,7.7,Mike Newell,"Joseph D. Pistone, Richard Woodley, Paul Attan...",77.0,"Al Pacino, Johnny Depp, Michael Madsen, Bruno ...","February 9, 2017 (India)",United States,"English, Italian, Japanese","$35,000,000 (estimated)","$124,909,762",2 hours 7 minutes,127,35000000.0,124909762,89909762.0
6,Finding Neverland,7.7,Marc Forster,"Allan Knee, David Magee",67.0,"Johnny Depp, Kate Winslet, Julie Christie, Rad...","January 28, 2005 (India)","United States, United Kingdom",English,"$25,000,000 (estimated)","$116,650,613",1 hour 46 minutes,106,25000000.0,116650613,91650613.0
7,Gonzo: The Life and Work of Dr. Hunter S. Thom...,7.6,Alex Gibney,"Alex Gibney, Hunter S. Thompson",73.0,"Hunter S. Thompson, Johnny Depp, Joe Cairo, Da...","July 18, 2008 (United States)",United States,"English, Portuguese",None,"$1,491,958",2 hours,120,NaN,1491958,NaN
8,When You're Strange,7.6,Tom DiCillo,Tom DiCillo,55.0,"Johnny Depp, John Densmore, Robby Krieger, Ray...","May 12, 2010 (United States)",United States,English,None,"$1,194,182",1 hour 26 minutes,86,NaN,1194182,NaN
9,Fear and Loathing in Las Vegas,7.5,Terry Gilliam,"Hunter S. Thompson, Terry Gilliam, Tony Grisoni",41.0,"Johnny Depp, Benicio Del Toro, Tobey Maguire, ...","May 22, 1998 (United States)",United States,English,"$18,500,000 (estimated)","$10,680,275",1 hour 58 minutes,118,18500000.0,10680275,-7819725.0


In [179]:
db.sql(
    "SELECT \"Director\", SUM(\"Worldwide Gross (usd)\") AS total_gross FROM imdb GROUP BY \"Director\" ORDER BY total_gross DESC LIMIT 10"
)

┌──────────────────────────┬─────────────┐
│         Director         │ total_gross │
│         varchar          │   int128    │
├──────────────────────────┼─────────────┤
│ James Cameron            │  8705319635 │
│ Steven Spielberg         │  7988558298 │
│ Anthony Russo, Joe Russo │  6721322058 │
│ Peter Jackson            │  6484962056 │
│ Christopher Nolan        │  6205459936 │
│ David Yates              │  5003353447 │
│ Jon Watts                │  3966372984 │
│ Joss Whedon              │  2966001713 │
│ J.J. Abrams              │  2924357232 │
│ Robert Zemeckis          │  2793892268 │
├──────────────────────────┴─────────────┤
│ 10 rows                      2 columns │
└────────────────────────────────────────┘

In [180]:
db.execute("ALTER TABLE imdb ADD COLUMN Year INT")
db.execute("ALTER TABLE imdb ADD COLUMN Decade INT")
db.execute(
    "UPDATE imdb SET Year = CAST(regexp_extract(\"Release Date\", '[0-9]{4}', 0) AS INT)"
)
db.execute(
    "UPDATE imdb SET Decade = (Year / 10)::INT * 10"
)
db.execute("SELECT \"Release Date\", Year, Decade FROM imdb LIMIT 5").df()

,Release Date,Year,Decade
0,"October 14, 1994 (India)",1994,1990
1,"November 8, 2024 (Japan)",2024,2020
2,1978 (India),1978,1980
3,"April 1, 1975 (Turkey)",1975,1980
4,"September 25, 2001 (India)",2001,2000


In [181]:
db.execute(
    "COPY (SELECT * FROM imdb) TO 'duckdb_parquet' (FORMAT PARQUET, PARTITION_BY Decade)"
)

IOException: IO Error: Directory "duckdb_parquet" is not empty! Enable OVERWRITE option to overwrite files

In [182]:
# TL;DR: DuckDB can attach MySQL, Postgres, and SQLite databases in addition to databases stored in its own format. This allows data to be read into DuckDB and moved between these systems in a convenient manner.
# https://duckdb.org/2024/01/26/multi-database-support-in-duckdb.html

In [183]:
# quotes usage example
# db.sql('''
#     select "Worldwide Gross" from imdb
# '''
# ).df()

# db.sql("""
#     select 'Worldwide Gross' from imdb
# """
# ).df()
# 
# db.sql("""
#     select 
#     'Worldwide Gross',
#     "Worldwide Gross"
#     from imdb
# """
# ).df()

# Window functions

In [190]:
db.sql('''
WITH exploded AS (
  SELECT 
    Title,
    TRIM(value['unnest']) as language,
    "Release Date",
    Languages as languages
  FROM df,
    UNNEST(STRING_SPLIT(Languages, ',')) as value
  WHERE value IS NOT NULL
),
cleaned_dates AS (
  SELECT 
    *,
    TRIM(REGEXP_REPLACE("Release Date", '\\(.+\\)', '')) as date_cleaned,
    COALESCE(
      TRY_STRPTIME(TRIM(REGEXP_REPLACE("Release Date", '\\(.+\\)', '')), '%B %d, %Y'), -- date format same as in pandas and polars but differse from spark
      TRY_STRPTIME(TRIM(REGEXP_REPLACE("Release Date", '\\(.+\\)', '')), '%Y'),
      TRY_STRPTIME(TRIM(REGEXP_REPLACE("Release Date", '\\(.+\\)', '')), '%B %Y')
    ) as date
  FROM exploded
),
with_lags AS (
  SELECT 
    *,
    LAG(date) OVER (PARTITION BY language ORDER BY date) as previous_movie_date,
    LAG(Title) OVER (PARTITION BY language ORDER BY date) as previous_movie_title
  FROM cleaned_dates
  WHERE date IS NOT NULL
),
with_diffs AS (
  SELECT 
    *,
    DATEDIFF('day', previous_movie_date, date) as previous_movie_diff,
    ROUND(CAST(DATEDIFF('day', previous_movie_date, date) AS FLOAT) / 365) as previous_movie_diff_years,
    ROW_NUMBER() OVER (PARTITION BY language ORDER BY DATEDIFF('day', previous_movie_date, date) DESC) as rn
  FROM with_lags
  WHERE previous_movie_date IS NOT NULL
)
SELECT 
  Title as title,
  date,
  previous_movie_title,
  previous_movie_date,
  previous_movie_diff,
  previous_movie_diff_years,
  language,
  languages
FROM with_diffs
WHERE rn = 1
ORDER BY language;
''').df()

,title,date,previous_movie_title,previous_movie_date,previous_movie_diff,previous_movie_diff_years,language,languages
0,Rabbit-Proof Fence,2002-02-21,Walkabout,1971-07-01,11193,31.0,Aboriginal,"English, Aboriginal"
1,Tsotsi,2005-12-23,Lethal Weapon 2,1990-01-01,5835,16.0,Afrikaans,"Zulu, Sotho, Xhosa, Afrikaans, English"
2,Triangle of Sadness,2022-09-18,Beasts of No Nation,2017-10-04,1810,5.0,Akan,"English, Swedish, German, French, Greek, Tagal..."
3,Vivement dimanche!,1983-08-10,The Ipcress File,1965-03-18,6719,18.0,Albanian,"French, Albanian"
4,Children of a Lesser God,1988-06-22,The Miracle Worker,1962-07-28,9461,26.0,American Sign Language,"English, American Sign Language"
...,...,...,...,...,...,...,...,...
122,Pride,2014-09-12,Becket,1964-03-11,18447,51.0,Welsh,"English, Welsh"
123,Io Capitano,2023-09-07,Biutiful,2010-10-22,4703,13.0,Wolof,"Wolof, French, Arabic, English"
124,Tsotsi,2005-12-23,The Lion King,1994-10-29,4073,11.0,Xhosa,"Zulu, Sotho, Xhosa, Afrikaans, English"
125,Obchod na korze,1965-10-08,Since You Went Away,1944-07-20,7750,21.0,Yiddish,"Slovak, Yiddish, German, Latin"
